# Flash Attention

[論文](https://arxiv.org/abs/2205.14135)

## 概要

Transformerは、入力のシーケンスが長いと処理が重くなってしまう

原因は、アテンションの計算時間とメモリ使用量が入力シーケンスの長さ（系列長）の2乗に比例するため

本論文では、メモリの読み書きを考慮してアテンションを再設計した**Flash Attention**を提案する

Flash Attentionは、データをブロックに分けSRAM内でアテンションを計算し、メモリ間の読み書き回数を削減する手法

この手法により、系列長1KのGPT-2の学習速度を3倍高速化し、系列長を広げることで性能も改善した

## 導入

![](image/fig1.png)

Transformerの処理が重い理由は、計算速度ではなくメモリの転送速度にある（メモリ律速）

GPUは、カーネルを実行するためにHBM（High Bandwidth Memory）からSRAMにデータを読み込み、実行後に書き込む

HBMとSRAM間の読み込みと書き込みが、計算速度と比べ遅い:

- HBM
    - 転送速度 1.5TB/s
    - サイズ 40GB
- SRAM
    - 転送速度 19TB/s
    - サイズ 20MB

標準的なアテンションは次のように計算できる:

$$
S = QK^{\top} \in \mathbb{R}^{N \times N}
$$

- $Q$: $N\times d$ 行列のクエリ
- $K^\top$: $N\times d$ 行列のキーの転置
- $S$: $N\times N$ 行列の生のアテンションスコア

$$
P = \text{softmax}(S) \in \mathbb{R}^{N \times N}
$$

- $P$: $N\times N$ 行列のアテンションスコア（要素の総和が1になるテンソル）

$$
O = PV \in \mathbb{R}^{N \times d}
$$

- $V$: $N\times d$ 行列のバリュー
- $O$: $N\times d$ 行列の最終的な出力（バリューの重み付き和）

標準的なアテンションの実装は、ソフトマックスの計算に全ての列の情報が必要

その結果HBMとSRAM間の読み込みと書き込みが多く発生してしまう:

1. HBMから $Q$ と $K$ をブロックごとにSRAMに **読み込み、** 生のアテンションスコア $S$ を計算し、HBMに **書き戻す**
1. HBMから $S$ をSRAMに **読み込み、** アテンションスコア $P$ を計算しHBMに **書き戻す**
1. HBMから $P$ と $V$ をブロックごとにSRAMに **読み込み、** $O$ を計算しHBMに **書き戻す**
1. Oを返す

![](image/algorithm0.png)


Flash Attentionは、低速なHBM（High Bandwidth Memory）と高速なSRAM間の読み書きが少なくなるように設計:

1. 順伝播では、入力をブロックに分割し（tiling）、SRAM内でブロックごとのアテンションを計算する（図参照）
2. 逆伝搬では、順伝播の中間計算結果（$S$と$P$）をHBMに保存せず、再計算する

標準的なテンションのHBMへの読み書き回数（IO複雑性）は、系列長の2乗に比例:

$$
\Omega(Nd + N^2)
$$

- $N$: 系列長
- $d$: アテンションヘッドの次元数

Flash AttentionのIO複雑性は、SRAMのサイズの逆数が乗算され、系列長に線形に比例し、最大9倍低くなる:

$$
O(N^2 d^2 M^{-1})
$$

- $M$: SRAMのサイズ

ベンチマークの結果:

- モデルの学習を高速化できた
    - 系列長512のBERT-largeの学習を15%高速化
    - 系列長1KのGPT-2の学習を3倍高速化
    - 系列長1Kから4Kのlong-range arenaの学習を2.4倍高速化
- モデルの品質を改善できた
    - GPT-2のパープレキシティを0.7改善
    - 長文分類の性能を6.4ポイント改善
- アテンション計算を高速化できた
    - 一般的な系列長で最大3倍高速

## Flash Attention

Flash Attentionアルゴリズムのポイントは、**タイリング** と **逆伝搬時の統計量の再計算** にある

### タイリング

Flash Attentionは、データをブロックに分け（タイリング）SRAM内でアテンションを計算する

SRAM内でアテンションを計算するため、入力全体ではなく部分的なソフトマックス計算が必要になる（オンラインソフトマックス）

部分的なソフトマックス計算のために、標準的なソフトマックス関数を **分解** する:

分解するために、ベクトル $x$ のソフトマックス計算に対して、3つの特徴量 $m(x)$・$f(x)$・$l(x)$ を定義

$m(x)$ は、$x$ の要素の **最大値** :

$$
m(x) := \max_{i} x_i
$$

$f(x)$ は、**ソフトマックスの分子** （オーバーフローを防ぐため各要素から最大値を引き、指数関数を適用したベクトル）:

$$
f(x) := [e^{x_1 - m(x)}, ..., e^{x_B - m(x)}
]
$$

$l(x)$ は、 **ソフトマックスの分母** （$f(x)$ の全要素の合計値）:

$$
l(x) := \sum_{i} f(x)_i
$$

$x$ の最終的なソフトマックスの出力:

$$
\text{softmax}(x) := \frac{f(x)}{l(x)}
$$

例えば、2つのベクトル $x^{(1)}, x^{(2)} \in \mathbb{R}^B$ を結合した $x =[x^{(1)} x^{(2)}] \in \mathbb{R}^{2B}$のソフトマックス計算は次のように分解できる:

$m(x)$は、結合したベクトルの要素の最大値:

$$
m(x) = m([x^{(1)} x^{(2)}]) = \max(m(x^{(1)}), m(x^{(2)}))
$$

$l(x)$は、結合したベクトルの$f(x)$の全要素の合計値（$x^{(1)}$と$x^{(2)}$のソフトマックスの分母をそれぞれの最大値で重み付けした和）:

$$
l(x) = l([x^{(1)} x^{(2)}]) = e^{m(x^{(1)}) - m(x)}l(x^{(1)}) + e^{m(x^{(2)}) - m(x)}l(x^{(2)})
$$

結合したベクトルの最終的なソフトマックスの出力:

$$
\text{softmax}(x) = \frac{f(x)}{l(x)}
$$

新しいベクトル（例えば $x^{(3)}$）が追加された場合は、計算済みの $m(x)$ と $l(x)$ を更新し、 $x^{(3)}$ を含めたソフトマックスを高速に計算できる

この仕組みを応用することで、ブロックごとにSRAM内でアテンションを計算できる（Algorithm 1）

### 再計算

標準的なアテンションの場合、逆伝搬時（$dQ$・$dK$・$dV$・$dO$ の計算時）に中間計算結果 $S$ （生のアテンションスコア）と $P$ （アテンションスコア）が必要

Flash Attentionでは、中間計算結果 $S$ と $P$ をHBMに保存せず、再計算する

生のアテンションスコア $S$ は、$Q$ と $K$ のブロックをSRAMに読み込み再計算する:

$$
S_{ij} = \tau Q_i K_j^T
$$

- $Q_i$: $i$ 番目のクエリ
- $K_j$: $j$ 番目のキー
- $\tau$ は、ソフトマックスのスケール定数

アテンションスコア $P$ は、該当ブロックの $m$ と $l$ から再計算できる:

$$
P_{ij} = \text{diag}(l_i)^{-1} \exp{(S_{ij} - m_i)}
$$

- $\text{diag}(l_i)^{-1}$: 対角行列の逆行列でソフトマックスの分母の逆数
- $\exp{(S_{ij} - m_i)}$: ソフトマックスの分子

再計算により $S$ と $P$ をHBMに保存する必要がなくなり、メモリ使用量とメモリアクセスを削減できる

### アルゴリズム

Flash Attentionは、小さなブロック単位でアテンションを計算し、メモリ使用量とメモリアクセスを削減する

大まかな流れは、 $K$ ブロックと $V$ ブロックをロードして固定し、 $Q$ ブロックを順に処理し、対応する $O$ ブロックを計算する

HBMに $Q$ ・ $K$ ・ $V$ があり、SRAMのサイズが $M$ とする

1. SRAMサイズ $M$ に基づいて、 $K$ ・ $V$ から読み込むブロックの列数 $B_c$ と、$Q$ から読み込むブロックの行数 $B_r$ を設定
2. HBMに、アテンションの計算結果 $O$・ソフトマックスの分母 $l$ ・生のアテンションスコアの最大値 $m$ を格納するメモリを作成
3. $Q$ ・ $K$ ・ $V$ のブロックサイズを決定
4. $O$ ・ $l$ ・ $m$ のブロックサイズを決定
5. $K_j$ ブロックと $V_j$ ブロックを順番に処理する外側のループを開始
6. HBMから$ K_j$ ブロック・ $V_j$ ブロックをSRAMに読み込む
7. $Q_i$ ブロックを順番に処理する内側のループを開始
8. $Q_i$ ブロック・ $O_i$ ブロック・ $l_i$ ブロック・ $m_i$ ブロックをSRAMに読み込む
9. 生のアテンションスコア $S_{ij} = Q_iK_j^T$ を計算
10. 生のアテンションスコアの要素の最大値 $\tilde{m}_{ij} = \text{rowmax}(S_{ij})$ を求め、アテンションスコア $\tilde{P}_{ij} = \exp{(S_{ij} - \tilde{m}_{ij})}$ を計算し、ソフトマックスの分母 $\tilde{l}_{ij} = \text{rowsum}(\tilde{P}_{ij})$ を計算
11. 過去の $m_i$ と $l_i$ と、$\tilde{m}_ij$ と $\tilde{l}_{ij}$ を使って、新しい$m_i^{\text{new}}$と$l_i^{\text{new}}$を計算
12. $m_i^{\text{new}}$ と $l_i^{\text{new}}$ で過去のアテンション $O_i$ をスケールして 、現在のブロックのアテンションを使って $O_i$ を更新し、HBMに書き戻す
13. 次のループで使用する $m_i^{\text{new}}$ と $l_i^{\text{new}}$ を $m_i$ と $l_i$ としてHBMに書き戻す
14. 内側のループ終了
15. 外側のループ終了
16. $O$を返す

![](image/algorithm1.png)

## Flash Attentionの順伝播の詳細

順伝播では、クエリ $Q$ ・キー $K$ ・バリュー $V$ からアテンション $O$ を求める:

$$
S = QK^T \in \mathbb{R}^{N\times N}
$$

$$
P = \text{softmax}(S) \in \mathbb{R}^{N\times N}
$$

$$
O = PV \in \mathbb{R}^{N\times d}
$$

ブロックごとの生のアテンションスコア $S_{ij}$ は次式で求められる:

$$
S_{ij} = q_i^T k_j
$$

- $q_i$: $i$番目のクエリブロック
- $k_j$: $j$番目のキーブロック

ソフトマックスの分母は次式で求められる:

$$
L_i = \sum_{j} e^{q_i^T k_j}
$$

$v_j$ を $j$ 番目のバリューブロックとすると、出力の $i$ 番目のブロックのアテンションは次式で求められる:

$$
o_i = P_{i:}V = \sum_{j} P_{ij}v_j = \sum_{j} \frac{e^{q_i^T k_j}}{L_i}v_j
$$

- $P_{ij}$: $i$ 番目のクエリブロックと $j$ 番目のキーブロックのアテンションスコア

以上の計算式で、$L_i$ を計算し、 $\frac{e^{q_i^T k_j}}{L_i}v_j$ を繰り返し足し合わせることで、$O(N)$の線形メモリで全てのアテンション $O$ を計算できる

因果マスクとドロップアウトを考慮した完全な順伝播アルゴリズム:

1. 乱数生成器の状態 $\mathcal{R}$ を初期化し、HBMに保存
2. SRAMサイズ $M$ に基づいて、$K$ ・$V$ から読み込むブロックごとの列数 $B_c$ と、Qから読み込むブロックごとの行数 $B_r$ を決定
3. $O$ ・ $l$ ・ $m$ をHBMに初期化
4. $Q$ ・ $K$ ・ $V$ のブロックサイズを決定
5. $O$ ・ $T$ ・ $m$ のブロックサイズを決定
6. $K_j$ ブロック・ $V_j$ ブロックを順番に処理する外側のループを開始
7. $K_j$ ブロック・ $V_j$ ブロックをSRAMに読み込む
8. $Q_i$ ブロックを順番に処理する内側のループ
9. HBMから $K_j$ ブロックと $V_j$ ブロックをSRAMに読み込む
10. 生のアテンションスコアを計算 $S_{ij} = \tau Q_j K_j$
11. 生のアテンションスコアに因果マスクを適用 $S_{ij}^{\text{masked}}$
12. 生のアテンションスコアの最大値 $\tilde{m_{ij}}$ を求め、アテンションスコア $\tilde{P_{ij}}$ とソフトマックスの分母 $\tilde{l}_{ij}$ を計算
13. 最大値を更新し $m_i^{\text{new}}$、ソフトマックスの分子を更新 $l_i^{new}$
14. アテンションスコアにドロップアウトを適用 $P_{ij}^{\text{\text{dropped}}}$
15. $l_i^{\text{new}}$ と $m_i^{\text{new}}$ でアテンション $O_i$ を更新し、HBMに書き戻す
16. $m_i^{\text{new}}$ と $l_i^{\text{new}}$ を $m_i$ と $l_i$ としてHBMに書き戻す
17. 内側のループ終了
18. 外側のループ終了
19. $O$ を返す

![](image/algorithm2.png)

## Flash Attentionの逆伝播詳細

逆伝搬では、出力の勾配からクエリ・キー・バリューの勾配を求める:

- $\phi$: スカラーの損失関数
- $dO \in \mathbb{R}^{n\times d}$: 出力の勾配 $\frac{\partial{\phi}}{\partial{O}}$
- $dQ \in \mathbb{R}^{n\times d}$: クエリの勾配 $\frac{\partial{\phi}}{\partial{Q}}$
- $dK \in \mathbb{R}^{n\times d}$: キーの勾配 $\frac{\partial{\phi}}{\partial{K}}$
- $dV \in \mathbb{R}^{n\times d}$: バリューの勾配 $\frac{\partial{\phi}}{\partial{V}}$


$dV$ は連鎖律より、$dV = P^T dO$ で求められる:

$$
dv_j = \sum_{i} P_{ij} do_i = \sum_{i} \frac{e^{q_i^T k_j}}{L_i} do_i
$$

- $L_i$ は、順伝播で計算済みのソフトマックスの分母
- $dv_j$ は、繰り返し足し合わせることで追加のメモリ無しで計算が可能

$dQ$ と $dK$ を求めるために、$dP$ と $dS$ が必要

$dP$ は $dP = dOV^T$ で求められる:

$$
dP_{ij} = do_i^T v_j
$$

$dS$は、$y=\text{softmax}(x)$のヤコビアンが$diag(y) - yy^T$という事実より求められる:

$$
dS_{i:} = (diag(P_{i:}) - P_{i:}^T P_{i:}) dP_{i:} = P_{i:} \circ dP_{i:} - (P_{i:}^T dP_{i:}) P_{i:}
$$

$$
D_i = P_{i:}^T dP_{i:} = \sum_{j} \frac{e^{q_i^T k_j}}{L_i} do_i^T v_j = do_i^T \sum_{j} \frac{e^{q_i^T k_j}}{L_i} v_j = do_i^T o_i
$$

$$
dS_{i:} = P_{i:} \circ dP_{i:} - D_i P_{i:}
$$

$dQ$ は、$S_{ij} = q_i^T k_j$ より計算できる:

$$
dq_i = \sum_{j} dS_{ij} k_j = \sum_{j} P_{ij}(dP_{ij} - D_i)k_j = \sum_{j} \frac{e^{q_i^T k_j}}{L_i}(do_i^T v_j - D_i)k_j
$$

$dK$も同様に求められる:

$$
dk_j = \sum_{i} dS_{ij} q_i = \sum_{i} P_{ij}(dP_{ij} - D_i)q_i = \sum_{i} \frac{e^{q_i^T k_j}}{L_i}(do_i^T v_j - D_i)q_i
$$

以上の計算式で、逆伝搬も $O(N)$ の線形メモリで計算できる

標準的な逆伝播アルゴリズム:

![](image/algorithm3.png)

Flash Attentionでの逆伝播アルゴリズム:

$K$と$V$のブロックをロードし、$Q$ブロックを順番に処理して、KとVの勾配を足し続ける

1. 順伝播で使用した乱数生成器の状態 $\mathcal{R}$ を復元
2. $K$・$V$から読み込むブロックの列数$B_c$と$Q$から読み込むブロックの行数$B_r$を設定
3. $Q$・$K$・$V$・のブロックサイズを決定
4. $O$・$l$・$m$のブロックサイズを決定
5. $dQ$・$dK$・$dV$を初期化し、ブロックサイズを決定
6. $K_j$ブロックと$V_j$ブロックを順番に処理する外側のループを開始
7. HBMから$K_j$・$V_j$をSRAMに読み込む
8. SRAMで$\tilde{dK_j}$と$\tilde{dV_j}$を0で初期化
9. $Q_i$ブロックを順番に処理する内側のループを開始
10. HBMから$Q_i$・$O_i$・$dO_i$・$dQ_i$・$l_i$・$m_i$を読み込む
11. 生のアテンションスコアを計算 $S_ij = \tau Q_i K_j^T$（$\tau$ は、ソフトマックスのスケール定数）
12. 因果マスクを適用 $S_{ij}^{\text{masked}} = \text{MASK}(S_{ij})$
13. アテンションスコアを計算 $P_{ij} = \text{diag}(l_i)^{-1} \exp{(S_{ij}^{\text{masked}} - m_i)}$
14. 乱数生成器からドロップアウト用のマスクを復元
15. アテンションスコアにドロップアウトを適用 $P_{ij}^{\text{dropped}} = P_{ij} Z_{ij}$
16. $\tilde{dV}$に$(P_{ij}^{\text{dropped}})^T dO_i$を加算
17. $dP_{ij}^{\text{dropped}} = dO_iV_j^T$を計算
18. $dP_{ij} = dP_{ij}^{\text{dropped}} Z_{ij}$を計算
19. $D_i = \text{rowsum}(dO_i O_i)$を計算
20. $dS_{ij} = P_{ij} (dP_{ij} - D_i)$を計算
21. $dQ_i$に$\tau dS_{ij}K_j$を加算し、HBMに書き込む
22. $\tilde{dK_j}$に$\tau dS_{ij}^T Q_i$を加算
23. 内側のループ終了
24. $\tilde{dK_j}$を$dK$として、$\tilde{dV_j}$を$dV_j$としてHBMに書き戻す
25. 外側のループ終了
26. $dQ$・$dK$・$dV$を返す

![](image/algorithm4.png)

## 実装

### 環境構築

In [1]:
%pip install triton tabulate pytest

import logging
import os
import tabulate
import torch
import triton
import triton.language as tl
from triton.tools.tensor_descriptor import TensorDescriptor
import pytest

# ログ設定

def custom_format(record):
    match record.levelno:
        case logging.DEBUG:
            level = '🟦'
        case logging.INFO:
            level = '🟩'
        case logging.WARNING:
            level = '🟨'
        case logging.ERROR:
            level = '🟥'
        case logging.CRITICAL:
            level = '🛑'
    return f"{level} {record.getMessage()}"

logger = logging.getLogger()
logger.setLevel(logging.NOTSET)

for handler in logger.handlers:
    logger.removeHandler(handler)

if os.path.exists('triton.log'):
    os.remove('triton.log')

formatter = logging.Formatter()
formatter.format = custom_format

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

file_handler = logging.FileHandler('triton.log')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# 3.4.0
logger.info(f'Tritonバージョン: {triton.__version__}')

DEVICE = triton.runtime.driver.active.get_active_torch_device()
logger.info(f'使用デバイス: {DEVICE}')

Note: you may need to restart the kernel to use updated packages.


🟩 Tritonバージョン: 3.4.0
🟩 使用デバイス: cuda:0


### ユーティリティ関数

In [2]:
def is_hip():
    return triton.runtime.driver.active.get_current_target().backend == "hip"

is_hip()

False

In [3]:
def is_cuda():
    return triton.runtime.driver.active.get_current_target().backend == "cuda"

is_cuda()

True

In [4]:
def supports_host_descriptor():
    return is_cuda() and torch.cuda.get_device_capability()[0] >= 9

supports_host_descriptor()

True

In [5]:
def is_blackwell():
    return is_cuda() and torch.cuda.get_device_capability()[0] == 10

is_blackwell()

False

In [6]:
def is_hopper():
    return is_cuda() and torch.cuda.get_device_capability()[0] == 9

is_hopper()

False

In [ ]:
# ディスクリプタの設定
# ディスクリプタは、HBM上の巨大なテンソルからデータブロックを読み込む際の形状

def _host_descriptor_pre_hook(nargs):
    BLOCK_M = nargs["BLOCK_M"]
    BLOCK_N = nargs["BLOCK_N"]
    HEAD_DIM = nargs["HEAD_DIM"]

    if not isinstance(nargs["desc_q"], TensorDescriptor):
        return

    # QのブロックをSRAMにロードする際の形状を (BLOCK_M, HEAD_DIM) に設定
    # BLOCK_M個のクエリベクトル全体（HEAD_DIM）を読み込む
    nargs["desc_q"].block_shape = [BLOCK_M, HEAD_DIM]

    # FP8を使用する場合は、メモリレイアウトが異なる
    if nargs["FP8_OUTPUT"]:
        nargs["desc_v"].block_shape = [HEAD_DIM, BLOCK_N]
    else:
        # VのブロックをSRAMにロードする際の形状を (BLOCK_N, HEAD_DIM) に設定
        # BLOCK_N個のバリューベクトル全体（HEAD_DIM）を読み込む
        nargs["desc_v"].block_shape = [BLOCK_N, HEAD_DIM]

    # KのブロックをSRAMにロードする際の形状を (BLOCK_N, HEAD_DIM) に設定
    # BLOCK_N個のキーベクトル全体（HEAD_DIM）を読み込む
    nargs["desc_k"].block_shape = [BLOCK_N, HEAD_DIM]

    # OのブロックをSRAMにロードする際の形状を (BLOCK_M, HEAD_DIM) に設定
    # BLOCK_M個の出力ベクトル全体（HEAD_DIM）を読み込む
    nargs["desc_o"].block_shape = [BLOCK_M, HEAD_DIM]

In [8]:
def keep(conf):
    BLOCK_M = conf.kwargs["BLOCK_M"]
    BLOCK_N = conf.kwargs["BLOCK_N"]
    return not (
        is_cuda() and \
        torch.cuda.get_device_capability()[0] == 9 and \
        BLOCK_M * BLOCK_N < 128 * 128 and \
        conf.num_warps == 8
    )

In [9]:
def prune_invalid_configs(configs, named_args, **kwargs):
    N_CTX = kwargs["N_CTX"]

    # Filter out configs where BLOCK_M > N_CTX
    return [conf for conf in configs if conf.kwargs.get("BLOCK_M", 0) <= N_CTX]

In [10]:
@triton.jit
def _maybe_make_tensor_desc(desc_or_ptr, shape, strides, block_shape):
    if isinstance(desc_or_ptr, tl.tensor_descriptor):
        return desc_or_ptr
    else:
        return tl.make_tensor_descriptor(desc_or_ptr, shape, strides, block_shape)

### 自動チューナーの設定

In [ ]:
# ソフトウェアパイプラインのステージ数
# 多いと並列実行のレイテンシが改善するが、メモリ使用量も増加する

if is_hip():
    NUM_STAGES_OPTIONS = [1]
elif supports_host_descriptor():
    NUM_STAGES_OPTIONS = [2, 3, 4]
else:
    NUM_STAGES_OPTIONS = [2, 3, 4]

NUM_STAGES_OPTIONS

[2, 3, 4]

In [ ]:
# ブロックごとのクエリの行数BMは64または128
# ブロックごとのキー/バリューの行数BNは32、64、128
# 一つのブロックを処理するために使用するワープ数は4または8
configs = [
    triton.Config({'BLOCK_M': BM, 'BLOCK_N': BN}, num_stages=s, num_warps=w, pre_hook=_host_descriptor_pre_hook) \
    for BM in [64, 128]\
    for BN in [32, 64, 128]\
    for s in NUM_STAGES_OPTIONS \
    for w in [4, 8]\
]

# テスト用の設定
if "PYTEST_VERSION" in os.environ:
    configs = [
        triton.Config(dict(BLOCK_M=128, BLOCK_N=64), num_stages=2, num_warps=4, pre_hook=_host_descriptor_pre_hook),
    ]

configs

### 順伝搬カーネル

#### _attn_fwd_inner

Algorithm 2の実装

**1つのクエリのブロック** に対して、キー・バリューの全ブロックを順に処理し、アキュームレータを更新するワーカーカーネル

アキュームレータ`acc`は、アテンションスコア $P$ とバリュー$V$ の行列積

ラッパーカーネル（`_attn_fwd`）で、アキュームレータをソフトマックスの分母$l$で割ることでアテンションを計算する

因果マスクを適用するため、アテンションスコアの対角成分上のブロックとそれ以外で異なる計算:

- STAGE == 1: アテンション行列の対角成分以外のブロック全てに対する処理（因果マスクを考慮しない）
- STAGE == 2: アテンション行列の対角成分のブロックの処理（因果マスクを考慮する）

アテンション行列と因果マスク（[ref](https://livebook.manning.com/wiki/categories/llm/causal+attention)）:

![](image/causal_attention.png)

※ Tritonではクエリ単位ではなく、ブロック単位（複数のクエリ）で処理を行うため図の解釈に注意

In [ ]:
@triton.jit
def _attn_fwd_inner(
    acc, # 出力を集約するアキュームレータ
    l_i, # 前ステップまでのソフトマックスの分母
    m_i, # 前ステップまでの生のアテンションスコアの要素の最大値
    q, # SRAMにロード済みのクエリのブロック (BLOCK_M, HEAD_DIM)
    desc_k, # HBM上のキーのディスクリプタ
    desc_v,  # HBM上のバリューのディスクリプタ
    offset_y, # ベースオフセット（特定のバッチの特定のヘッドのデータの開始位置）
    dtype: tl.constexpr, # 計算に使用するデータ型
    start_m, # Qブロックの開始行インデックス
    qk_scale, # QKに掛けるスケーリング係数
    BLOCK_M: tl.constexpr, # Qブロックの行数
    HEAD_DIM: tl.constexpr, # アテンションヘッドの次元数
    BLOCK_N: tl.constexpr, # KブロックもしくはVブロックの行数
    STAGE: tl.constexpr, # 因果マスキングのステージ指定
    offs_m: tl.constexpr, # Qブロックの行インデックスの配列 (BLOCK_M,)
    offs_n: tl.constexpr, # Kブロックの列インデックスの配列 (BLOCK_N,)
    N_CTX: tl.constexpr, # シーケンスの長さ（コンテキスト長） N
    warp_specialize: tl.constexpr, # ワープ単位での最適化のフラグ
    IS_HOPPER: tl.constexpr # NVIDIA Hopperアーキテクチャのフラグ
):

    #################
    # ポインタの初期化 #
    #################

    # アテンション行列の対角成分より前の部分を処理する場合（off-diagonal）
    if STAGE == 1:
        # Qのブロックよりも前の位置にあるKとVのブロックが計算範囲
        lo, hi = 0, start_m * BLOCK_M

    # アテンション行列の対角成分に対する処理する場合（on-diagonal）
    elif STAGE == 2:
        # 現在のQブロックと同じ位置にあるKとVのブロックが計算範囲
        lo, hi = start_m * BLOCK_M, (start_m + 1) * BLOCK_M
        lo = tl.multiple_of(lo, BLOCK_M)

    # 因果マスキングが不要な場合
    else:
        # シーケンス全体にわたるKとVのブロックが計算範囲
        lo, hi = 0, N_CTX

    # Kブロックのオフセットを計算
    offsetk_y = offset_y + lo

    # Vブロックのオフセットを計算
    # 注意: FP8データ型の場合、Vが転置されているためオフセット計算が異なる
    if dtype == tl.float8e5:
        offsetv_y = offset_y * HEAD_DIM + lo
    else:
        offsetv_y = offset_y + lo

    #######################################
    # KブロックとVブロックを順番に処理するループ #
    #######################################

    # ポインタの開始位置をBLOCK_Nずつ勧めながら、KとVのブロックを順番に処理
    for start_n in tl.range(lo, hi, BLOCK_N, warp_specialize=warp_specialize):

        # コンパイラへのヒント（KとVのブロックの開始行のインデックスはBLOCK_Nの倍数）
        start_n = tl.multiple_of(start_n, BLOCK_N)

        # KのブロックをSRAMにロードし、転置
        # (BLOCK_N, HEAD_DIM) -> (HEAD_DIM, BLOCK_N)
        k = desc_k.load([offsetk_y, 0]).T

        # 生のアテンションスコアQKを計算
        # S_ij = Qi_K_j^T
        # (BLOCK_M, HEAD_DIM) @ (HEAD_DIM, BLOCK_N) -> (BLOCK_M, BLOCK_N)
        qk = tl.dot(q, k)

        # アテンション行列の対角の場合（on-diagonal）
        if STAGE == 2:

            # 「Qの行インデックス >= Kの列インデックス」を満たす2次元因果マスクを作成
            mask = offs_m[:, None] >= (start_n + offs_n[None, :])

            # 生のアテンションスコアQKのスケールを調整し、マスクを適用
            # S_ij^{masked} = MASK(\tau S_{ij})
            # (BLOCK_M, BLOCK_N)
            qk = qk * qk_scale + tl.where(mask, 0, -1.0e6)

            # 調整したアテンションスコアQKの要素の最大値を更新
            # m_i^{new} = max(m_i, rowmax(S_ij^{masked}))
            # (BLOCK_M,)
            m_ij = tl.maximum(m_i, tl.max(qk, 1))

            # 調整したアテンションスコアから最大値を引く
            # S_ij = S_ij^{masked} - m_i^{new}
            # (BLOCK_M, BLOCK_N) - (BLOCK_M, 1) -> (BLOCK_M, BLOCK_N)
            qk -= m_ij[:, None]

        # アテンション行列の対角成分より前の部分の場合（off-diagonal）
        else:
            # 生のアテンションスコアQKのスケールを調整し、最大値を更新
            # m_i^{new} = max(m_i, rowmax(τ S_ij) * \tau)
            # (BLOCK_M,)
            m_ij = tl.maximum(m_i, tl.max(qk, 1) * qk_scale)

            # 調整したアテンションスコアから最大値を引く
            # S_ij = S_ij - m_i^{new}
            # (BLOCK_M, BLOCK_N) - (BLOCK_M, 1) -> (BLOCK_M, BLOCK_N)
            qk = qk * qk_scale - m_ij[:, None]

        # アテンションスコアPを計算（指数ではなく2のべき乗を使用することで高速化）
        # \tilde{P}_{ij} = \exp(S_{ij})
        # (BLOCK_M, BLOCK_N)
        p = tl.math.exp2(qk)

        # 最大値が更新されたため、再スケーリング用のアルファを計算（e^{m_i-m_i^{new}}
        # alpha = exp(m_i - m_i^{new})
        # (BLOCK_M,)
        alpha = tl.math.exp2(m_i - m_ij)

        # ソフトマックスの分母を計算（アテンションスコアを列方向に潰して合計）
        # \tilde{l}_{ij} = rowsum(\tilde{P}_{ij})
        # (BLOCK_M,)
        l_ij = tl.sum(p, 1)

        # 出力のアキュームレータを更新
        if not IS_HOPPER and warp_specialize and BLOCK_M == 128 and HEAD_DIM == 128:
            BM: tl.constexpr = acc.shape[0]
            BN: tl.constexpr = acc.shape[1]
            acc0, acc1 = acc.reshape([BM, 2, BN // 2]).permute(0, 2, 1).split()
            acc0 = acc0 * alpha[:, None]
            acc1 = acc1 * alpha[:, None]
            acc = tl.join(acc0, acc1).permute(0, 2, 1).reshape([BM, BN])
        else:
            # 過去に計算したアキュームレータをアルファでスケーリング（新しい最大値の基準に合わせる）
            # O_i = O_i * alpha
            acc = acc * alpha[:, None]

        # VブロックをSRAMにロード
        # FP8データ型の場合はメモリレイアウトが異なる（Blockwell世代以前）
        if dtype == tl.float8e5:
            # (HEAD_DIM, BLOCK_N) -> (BLOCK_N, HEAD_DIM)
            v = desc_v.load([0, offsetv_y]).T
        else:
            # (BLOCK_N, HEAD_DIM)
            v = desc_v.load([offsetv_y, 0])

        # アテンションスコアPをキャスト
        p = p.to(dtype)

        # PとVの内積を計算し、アキュームレータに加算
        # O_i += \tilde{P}_{ij} V_j
        # (BLOCK_M, BLOCK_N) @ (BLOCK_N, HEAD_DIM) -> (BLOCK_M, HEAD_DIM)
        acc = tl.dot(p, v, acc)

        ###################
        # 次のループへの準備 #
        ###################

        # ソフトマックスの分母を更新
        # l_i^{new} = l_i * alpha + \tilde{l}_{ij}
        l_i = l_i * alpha + l_ij

        # 最大値を更新
        m_i = m_ij

        # Kのブロックのオフセットを更新
        offsetk_y += BLOCK_N

        # Vのブロックのオフセットを更新
        offsetv_y += BLOCK_N

    return acc, l_i, m_i

#### _attn_fwd

**1つのクエリブロックに対する** 全てのアテンション計算を実行するマネージャー

担当するブロックのクエリテンソルを読み込み、STAGEを決定し、_attn_fwd_innerを呼び出し、アテンションの計算を完成させる

因果マスクを適応する場合、対角成分以外のブロックを処理し、対角成分のブロックを続けて処理する

In [ ]:
# 自動チューナーの設定
@triton.autotune(
    configs=list(filter(keep, configs)),
    key=["N_CTX", "HEAD_DIM", "FP8_OUTPUT", "warp_specialize"],
    prune_configs_by={'early_config_prune': prune_invalid_configs}
)
@triton.jit
def _attn_fwd(
    sm_scale, # QKに掛けるスケーリング係数
    M, # 生のアテンションスコアの要素の最大値mへのポインタ
    Z, # バッチサイズ
    H, # ヘッド数
    desc_q, # HBM上のQのディスクリプタ
    desc_k, # HBM上のKのディスクリプタ
    desc_v, # HBM上のVのディスクリプタ
    desc_o, # HBM上のOのディスクリプタ
    N_CTX, # シーケンスの長さ（コンテキスト長）
    HEAD_DIM: tl.constexpr, # アテンションヘッドの次元数 d
    BLOCK_M: tl.constexpr, # Qブロックの行数 B_r
    BLOCK_N: tl.constexpr, # KブロックもしくはVブロックの列数 B_c
    FP8_OUTPUT: tl.constexpr, # FP8データ型を使用するフラグ
    STAGE: tl.constexpr, # 因果マスクを使う場合は3、使わない場合は1
    warp_specialize: tl.constexpr, # ワープの処理を最適化するフラグ
    IS_HOPPER: tl.constexpr, # NVIDIA Hopperアーキテクチャのフラグ
):

    #########
    # 初期化 #
    #########

    dtype = tl.float8e5 if FP8_OUTPUT else tl.float16

    tl.static_assert(BLOCK_N <= HEAD_DIM)

    # 現在のプログラムIDの0次元目を取得し、Qのブロックのインデックスとする
    start_m = tl.program_id(0)

    # 現在のプログラムIDの1次元目を取得し、ヘッドとバッチの複合インデックスとする
    off_hz = tl.program_id(1)

    # 現在のプログラムインスタンスが担当するバッチインデックスを計算
    off_z = off_hz // H

    # 現在のプログラムインスタンスが担当するヘッドインデックスを計算
    off_h = off_hz % H

    # バッチとヘッド全体でのトークンの総数（平坦化に必要）
    y_dim = Z * H * N_CTX

    ####################
    # ディスクリプタを作成 #
    ####################

    # HBM上のQにアクセスするディスクリプタを作成
    desc_q = _maybe_make_tensor_desc(
        desc_q,
        shape=[y_dim, HEAD_DIM], # テンソルの形状
        strides=[HEAD_DIM, 1], # メモリ上でのデータの並び順
        block_shape=[BLOCK_M, HEAD_DIM] # SRAMにロードするブロックの形状（BLOCK_M個のクエリベクトル全体を読み込む）
    )

    # HBM上のVにアクセスするディスクリプタを作成
    # FP8を使用する場合は、Vが転置されている
    if FP8_OUTPUT:
        desc_v = _maybe_make_tensor_desc(
            desc_v,
            shape=[HEAD_DIM, y_dim], # テンソルの形状
            strides=[N_CTX, 1], # メモリ上でのデータの並び順
            block_shape=[HEAD_DIM, BLOCK_N] # SRAMにロードするブロックの形状
        )
    else:
        desc_v = _maybe_make_tensor_desc(
            desc_v, # HBM上のVにアクセスするディスクリプタ
            shape=[y_dim, HEAD_DIM], # テンソルの形状
            strides=[HEAD_DIM, 1], # メモリ上でのデータの並び順
            block_shape=[BLOCK_N, HEAD_DIM] # SRAMにロードするブロックの形状（BLOCK_N個のバリューベクトル全体を読み込む）
        )

    # HBM上のKにアクセスするディスクリプタを作成
    desc_k = _maybe_make_tensor_desc(
        desc_k,
        shape=[y_dim, HEAD_DIM], # テンソルの形状
        strides=[HEAD_DIM, 1], # メモリ上でのデータの並び順
        block_shape=[BLOCK_N, HEAD_DIM] # SRAMにロードするブロックの形状（BLOCK_N個のキーベクトル全体を読み込む）
    )

    # HBM上のOにアクセスするディスクリプタを作成
    desc_o = _maybe_make_tensor_desc(
        desc_o,
        shape=[y_dim, HEAD_DIM], # テンソルの形状
        strides=[HEAD_DIM, 1], # メモリ上でのデータの並び順
        block_shape=[BLOCK_M, HEAD_DIM] # SRAMにロードするブロックの形状（BLOCK_M個の出力ベクトル全体を読み込む）
    )

    ######################
    # 変数とポインタの初期化 #
    ######################

    # 担当するベースオフセットを計算
    # バッチオフセット * シーケンス長 * ヘッド数 + ヘッドオフセット * シーケンス長
    offset_y = off_z * (N_CTX * H) + off_h * N_CTX

    # 担当するQブロックのオフセットを計算
    # ベースオフセット + Qブロックのインデックス * Qブロックの行数
    qo_offset_y = offset_y + start_m * BLOCK_M

    # Qブロックの行インデックスの配列を作成
    # (BLOCK_M,)
    offs_m = start_m * BLOCK_M + tl.arange(0, BLOCK_M)

    # KブロックもしくはVブロックの列インデックスの配列を作成
    # (BLOCK_N,)
    offs_n = tl.arange(0, BLOCK_N)

    # 生のアテンションスコアの最大値をマイナス無限大で初期化
    # (BLOCK_M,)
    m_i = tl.zeros([BLOCK_M], dtype=tl.float32) - float("inf")

    # ソフトマックスの分母を1.0で初期化
    # (BLOCK_M,)
    l_i = tl.zeros([BLOCK_M], dtype=tl.float32) + 1.0

    # 出力のアキュームレータを0で初期化
    # (BLOCK_M, HEAD_DIM)
    acc = tl.zeros([BLOCK_M, HEAD_DIM], dtype=tl.float32)

    ################################
    # Qのロードとメインループの呼び出し #
    ################################

    # ソフトマックスのスケーリング係数を計算（指数ではなく2のべき乗を使用して高速化）
    qk_scale = sm_scale # 0.5
    qk_scale *= 1.44269504 # 1/log(2)

    # 担当するQブロックをHBMからSRAMに読み込み
    # (BLOCK_M, HEAD_DIM)
    q = desc_q.load([qo_offset_y, 0])

    # STAGEが1もしくは3の場合（0b01 & 0b01 || 0b11 & 0b01）
    if STAGE & 1:
        # 1の場合、因果マスクなしで全て計算
        # 3の場合、アテンション行列の対角成分より以外の部分に対する計算（off-diagonal）
        acc, l_i, m_i = _attn_fwd_inner(
            acc,
            l_i,
            m_i,
            q,
            desc_k,
            desc_v,
            offset_y,
            dtype,
            start_m,
            qk_scale,
            BLOCK_M,
            HEAD_DIM,
            BLOCK_N,
            4 - STAGE, # STAGEが1なら3、STAGEが3なら1を渡す
            offs_m,
            offs_n,
            N_CTX,
            warp_specialize,
            IS_HOPPER
        )

    # STAGEが3の場合（0b11 & 0b10）
    if STAGE & 2:
        # アテンション行列の対角成分に対する計算（on-diagonal）
        acc, l_i, m_i = _attn_fwd_inner(
            acc,
            l_i,
            m_i,
            q,
            desc_k,
            desc_v,
            offset_y,
            dtype,
            start_m,
            qk_scale,
            BLOCK_M,
            HEAD_DIM,
            BLOCK_N,
            2, # STAGEに2を渡す
            offs_m,
            offs_n,
            N_CTX,
            warp_specialize,
            IS_HOPPER
        )

    ###################
    # 最終処理と書き出し #
    ###################

    # 生のアテンションスコアの要素の最大値m_iを、数値的な安定性のためにlog-sum-expに変換
    # log-sum-exp = m_i + log(l_i)
    # (BLOCK_M,)
    m_i += tl.math.log2(l_i)

    # アテンションを計算
    # O_i = ソフトマックスの分子 / ソフトマックスの分母
    acc = acc / l_i[:, None]

    # m_iをHBMに書き出すポインタを計算
    # Mのポインタ + ヘッドとバッチのオフセット * シーケンス長 + Qブロックのオフセット
    # (BLOCK_M,)
    m_ptrs = M + off_hz * N_CTX + offs_m

    # m_iをHBMに書き出し
    tl.store(m_ptrs, m_i)

    # アテンションをHBMに書き出し
    # (BLOCK_M, HEAD_DIM)
    desc_o.store([qo_offset_y, 0], acc.to(dtype))

### 逆伝播カーネル

Algorithm 4を実装

#### _attn_bwd_preprocess

ソフトマックスの勾配計算に必要なDelta（$D_i$）を求めるワーカーカーネル

$$
D_i = \text{rowsum}(dO_i O_i)
$$

In [ ]:
@triton.jit
def _attn_bwd_preprocess(
    O, # 順伝播の出力テンソル
    DO, # Oの勾配テンソル
    Delta, # ソフトマックスの勾配計算に必要な中間テンソルへのポインタ
    Z, # バッチサイズ
    H, # ヘッド数
    N_CTX, # シーケンス長（コンテキスト長）
    BLOCK_M: tl.constexpr, # このカーネルが処理する行数
    HEAD_DIM: tl.constexpr # アテンションヘッドの次元数
):
    #################
    # ポインタの初期化 #
    #################

    # 現在のプログラムIDの0次元目を取得し、行インデックスのオフセットを計算
    # (BLOCK_M,)
    off_m = tl.program_id(0) * BLOCK_M + tl.arange(0, BLOCK_M)

    # 現在のプログラムIDの1次元目を取得し、ヘッドとバッチの複合インデックスとする
    off_hz = tl.program_id(1)

    # 列方向のインデックスを作成
    # (HEAD_DIM,)
    off_n = tl.arange(0, HEAD_DIM)

    ################ 
    # データのロード #
    ################ 

    # 順伝播の結果の一部をロード
    # ベースオフセット + バッチとヘッドのオフセット + 行オフセット + 列オフセット
    # (BLOCK_M, HEAD_DIM)
    o = tl.load(
        O + \
        off_hz * HEAD_DIM * N_CTX + \
        off_m[:, None] * HEAD_DIM + \
        off_n[None, :]
    )

    # 順伝播の結果の勾配の一部をロード
    # ベースオフセット + バッチとヘッドのオフセット + 行オフセット + 列オフセット
    # (BLOCK_M, HEAD_DIM)
    do = tl.load(
        DO + \
        off_hz * HEAD_DIM * N_CTX + \
        off_m[:, None] * HEAD_DIM + \
        off_n[None, :] \
    ).to(tl.float32)

    ##############
    # Deltaの計算 #
    ##############

    # oとdoを要素ごとに乗算し、列方向を潰して合計を計算
    # (BLOCK_M, HEAD_DIM) * (BLOCK_M, HEAD_DIM) -> (BLOCK_M,)
    delta = tl.sum(o * do, axis=1)

    # HBM上のDeltaに書き出し
    # ベースオフセット + バッチとヘッドのオフセット + 行オフセット
    tl.store(Delta + off_hz * N_CTX + off_m, delta)

#### _attn_bwd_dkdv

Qブロックに対するキーブロックの勾配 $dK$ ・バリューブロックの勾配 $dV$ を計算するワーカー関数

**SRAMにロード済みの$K$ブロックと$V$ブロックを固定し**、$Q$ブロックと$dO$ブロックを順に処理

In [ ]:
@triton.jit
def _attn_bwd_dkdv(
    dk, # Kの勾配を累積する変数
    dv, # Vの勾配を累積する変数
    Q, # 順伝播で使用したクエリテンソルへのポインタ
    k, # SRAMにロード済みのKブロック
    v, # SRAMにロード済みのVブロック
    sm_scale, # 順伝播で使用したスケーリング係数（1/log(2)）
    DO, # 出力Oの勾配テンソルへのポインタ
    M, # 順伝播の最後に保存したMテンソルへのポインタ
    D, # 事前計算されたDeltaテンソルへのポインタ
    stride_tok, # 次のトークン（次の行）に進むためのストライド（Q・K・V・DO共通）
    stride_d, # 次の次元（次の列）に進むためのストライド（Q・K・V・DO共通）
    H, # ヘッド数
    N_CTX, # シーケンス長（コンテキスト長）
    BLOCK_M1: tl.constexpr, # このカーネルで使用するQブロックの行数
    BLOCK_N1: tl.constexpr, # このカーネルで使用するKブロックもしくはVブロックの列数
    HEAD_DIM: tl.constexpr, # ヘッドの次元数
    start_n, # 担当するKブロックの開始列インデックス
    start_m, # 担当するQブロックの開始行インデックス
    num_steps, # forループの反復回数
    MASK: tl.constexpr # 因果マスキングを適用するかどうかのフラグ
):
    #################
    # ポインタを初期化 #
    #################

    # 現在のプログラムインスタンスが処理するQブロックの行インデックス範囲を計算
    # (BLOCK_M1,)
    offs_m = start_m + tl.arange(0, BLOCK_M1)

    # 現在のプログラムインスタンスが処理するKブロックの列インデックス範囲を計算
    # (BLOCK_N1,)
    offs_n = start_n + tl.arange(0, BLOCK_N1)

    # ヘッダの次元インデックスの配列を作成
    # (HEAD_DIM,)
    offs_k = tl.arange(0, HEAD_DIM)

    # 現在のプログラムインスタンスが処理するQブロックのポインタを計算
    # ベースポインタ + 行オフセット * トークンストライド + 列オフセット * 次元ストライド
    # (BLOCK_M1, HEAD_DIM)
    qT_ptrs = Q + \
        offs_m[None, :] * stride_tok + \
        offs_k[:, None] * stride_d

    # 現在のプログラムインスタンスが処理するDOブロックのポインタを計算
    # ベースポインタ + 行オフセット * トークンストライド + 列オフセット * 次元ストライド
    # (BLOCK_M1, HEAD_DIM)
    do_ptrs = DO + \
        offs_m[:, None] * stride_tok + \
        offs_k[None, :] * stride_d

    # BLOCK_N1はBLOCK_M1の倍数であることを検証
    tl.static_assert(BLOCK_N1 % BLOCK_M1 == 0)

    #########################
    # QとDOのブロックを順に処理 #
    #########################

    curr_m = start_m
    step_m = BLOCK_M1
    for blk_idx in range(num_steps):

        ##########################
        # アテンションスコアの再計算 #
        ##########################

        # QブロックをSRAMにロード
        # (BLOCK_M1, HEAD_DIM)
        qT = tl.load(qT_ptrs)

        # 生のアテンンションスコアの要素の最大値m_iへのポインタを計算
        # (BLOCK_M1,)
        offs_m = curr_m + tl.arange(0, BLOCK_M1)

        # m_iをSRAMにロード
        # (BLOCK_M1,)
        m = tl.load(M + offs_m)

        # 生のアテンションスコアを再計算 S_ij = Q_i K_j^T
        # (BLOCK_M1, BLOCK_N1) = (BLOCK_M1, HEAD_DIM) @ (HEAD_DIM, BLOCK_N1)
        qkT = tl.dot(k, qT)

        # アテンションスコアPを再計算 P_ij = exp(S_ij - m_i)
        # (BLOCK_M1, BLOCK_N1)
        pT = tl.math.exp2(qkT - m[None, :])

        ############
        # 勾配の計算 #
        ############

        # アテンションスコアにマスクを適用
        if MASK:
            mask = (offs_m[None, :] >= offs_n[:, None])
            pT = tl.where(mask, pT, 0.0)

        # DOブロックをSRAMにロード
        # (BLOCK_M1, HEAD_DIM)
        do = tl.load(do_ptrs)

        # pTをコピーしてfp16にキャスト
        # (BLOCK_M1, BLOCK_N1)
        ppT = pT
        ppT = ppT.to(tl.float16)

        # dVを累積
        # \tilde{dV_j} = \tilde{dV_j} + P_{ij}^T dO_i
        # (BLOCK_N1, BLOCK_M1) @ (BLOCK_M1, HEAD_DIM) = (BLOCK_N1, HEAD_DIM)
        dv += tl.dot(ppT, do)

        # デルタテンソル D_i をSRAMにロード
        # (BLOCK_N1,)
        Di = tl.load(D + offs_m)

        # dPを計算
        # dP_{ij} = dO @ V^T
        dpT = tl.dot(v, tl.trans(do)).to(tl.float32)

        # dSを計算
        # dS_{ij} = P_{ij} (dP_{ij} - D_i)
        dsT = pT * (dpT - Di[None, :])
        dsT = dsT.to(tl.float16)

        # dKを計算し、アキュームレータに加算
        # \tilde{dK_j} = \tilde{dK_j} + dS_{ij}^T Q_i
        # (HEAD_DIM, BLOCK_N1) @ (BLOCK_N1, BLOCK_M1) = (HEAD_DIM, BLOCK_M1)
        dk += tl.dot(dsT, tl.trans(qT))

        ################
        # ポインタの更新 #
        ################

        curr_m += step_m
        qT_ptrs += step_m * stride_tok
        do_ptrs += step_m * stride_tok

    return dk, dv

#### _attn_bwd_dq

`_attn_bwd_dq`は、$dQ$（クエリの勾配）を計算するワーカー関数

**SRAMにロード済みの一つのQブロックを固定し**、KブロックとVブロックをループして計算した勾配をアキュームレータに加算する

In [ ]:
@triton.jit
def _attn_bwd_dq(
    dq, # Qの勾配のアキュームレータ
    q, # SRAMにロード済みのQブロック
    K, # Kテンソル全体へのポインタ
    V, # Vテンソル全体へのポインタ
    do, # qに対応する出力Oの勾配ブロック
    m, # qに対応する生のアテンションスコアの要素の最大値mのブロック
    D, # 事前計算済みのDeltaブロックへのポインタ
    stride_tok, # 次のトークン（次の行）に進むためのストライド（Q・K・V・DO共通）
    stride_d, # 次の次元（次の列）に進むためのストライド（Q・K・V・DO共通）
    H, # ヘッド数
    N_CTX, # シーケンス長（コンテキスト長）
    BLOCK_M2: tl.constexpr, # このカーネルで使用するブロックサイズ
    BLOCK_N2: tl.constexpr, # このカーネルで使用するブロックサイズ
    HEAD_DIM: tl.constexpr, # ヘッドの次元数
    start_m, # 担当するQブロックの開始行インデックス
    start_n, # 担当するKブロックの開始列インデックス
    num_steps, # forループの反復回数
    MASK: tl.constexpr # 因果マスキングを適用するかどうかのフラグ
):
    #################
    # ポインタの初期化
    #################

    # 現在のプログラムインスタンスが処理するQブロックの行インデックス範囲を計算
    # (BLOCK_M2,)
    offs_m = start_m + tl.arange(0, BLOCK_M2)

    # 現在のプログラムインスタンスが処理するKブロックの列インデックス範囲を計算
    # (BLOCK_N2,)
    offs_n = start_n + tl.arange(0, BLOCK_N2)

    # ヘッダの次元インデックスの配列を作成
    # (HEAD_DIM,)
    offs_k = tl.arange(0, HEAD_DIM)

    # 現在のプログラムインスタンスが処理するKブロックのポインタを計算
    # ベースポインタ + 行オフセット * トークンストライド + 列オフセット * 次元ストライド
    # (HEAD_DIM, BLOCK_N2)
    kT_ptrs = K + \
        offs_n[None, :] * stride_tok + \
        offs_k[:, None] * stride_d

    # 現在のプログラムインスタンスが処理するVブロックのポインタを計算
    # ベースポインタ + 行オフセット * トークンストライド + 列オフセット * 次元ストライド
    # (HEAD_DIM, BLOCK_N2)
    vT_ptrs = V + \
        offs_n[None, :] * stride_tok + \
        offs_k[:, None] * stride_d

    # 現在のプログラムインスタンスが処理するDブロックをSRAMにロード
    # 注意: D（Delta）は事前にds_scaleで割られている
    # (BLOCK_M2,)
    Di = tl.load(D + offs_m)

    # BLOCK_M2はBLOCK_N2の倍数であることを検証
    tl.static_assert(BLOCK_M2 % BLOCK_N2 == 0)

    ########################
    # KとVのブロックを順に処理 #
    ########################

    curr_n = start_n
    step_n = BLOCK_N2
    for blk_idx in range(num_steps):

        # KブロックをSRAMにロード
        # (HEAD_DIM, BLOCK_N2)
        kT = tl.load(kT_ptrs)

        # VブロックをSRAMにロード
        # (HEAD_DIM, BLOCK_N2)
        vT = tl.load(vT_ptrs)

        # 生のアテンションスコアを再計算 S_ij = Q_i K_j^T
        # (BLOCK_M2, HEAD_DIM) @ (HEAD_DIM, BLOCK_N2) -> (BLOCK_M2, BLOCK_N2)
        qk = tl.dot(q, kT)

        # アテンションスコアPを再計算 P_ij = exp(S_ij - m_i)
        # (BLOCK_M2, BLOCK_N2)
        p = tl.math.exp2(qk - m)

        # 因果マスクを使用する場合
        if MASK:
            # マスクを作成
            offs_n = curr_n + tl.arange(0, BLOCK_N2)
            mask = (offs_m[:, None] >= offs_n[None, :])

            # マスクを適用
            p = tl.where(mask, p, 0.0)

        # アテンションスコアの勾配dPを計算
        # dP_{ij} = dO_{i} V_j^T
        dp = tl.dot(do, vT).to(tl.float32)

        # アテンションスコアの勾配dSを計算
        # dS = P * (dP - D_i)
        ds = p * (dp - Di[:, None])
        ds = ds.to(tl.float16)

        # Qの勾配dQを計算し、アキュームレータに加算
        # dQ_i = dQ_i + dS_{ij} K_j
        # 注意: kTは事前にqk_scaleでスケーリングされているため戻す必要がある
        dq += tl.dot(ds, tl.trans(kT))

        ################
        # ポインタの更新 #
        ################

        # Kブロックの列オフセットを更新
        curr_n += step_n

        # Kブロックのポインタを更新
        kT_ptrs += step_n * stride_tok

        # Vブロックのポインタを更新
        vT_ptrs += step_n * stride_tok

    return dq

#### _attn_bwd

逆伝搬全体を制御するマネージャー

担当範囲を決定し、必要なデータを準備し、ワーカー関数を呼び出し、$dQ$ ・ $dK$ ・ $dV$ をHBMに書き戻す

In [ ]:
@triton.jit
def _attn_bwd(
    Q, # 順伝播で使用したクエリテンソルへのポインタ
    K, # 順伝播で使用したキーテンソルへのポインタ
    V, # 順伝播で使用したバリューテンソルへのポインタ
    sm_scale, # 順伝播で使用したスケーリング係数（1/log(2)）
    DO, # 出力Oの勾配テンソル全体へのポインタ
    DQ, # 計算結果を書き込むQの勾配テンソル全体へのポインタ
    DK, # 計算結果を書き込むKの勾配テンソル全体へのポインタ
    DV, # 計算結果を書き込むVの勾配テンソル全体へのポインタ
    M, # 順伝播で保存した生のアテンションスコアの要素の最大値mへのポインタ
    D, # 事前計算されたDeltaテンソル
    stride_z, # バッチ方向に進むためのストライド（Q・K・V・DO共通）
    stride_h, # ヘッド方向に進むためのストライド（Q・K・V・DO共通）
    stride_tok, # 次のトークン（次の行）に進むためのストライド（Q・K・V・DO共通）
    stride_d, # 次の次元（次の列）に進むためのストライド（Q・K・V・DO共通）
    H, # ヘッド数
    N_CTX, # シーケンス長（コンテキスト長）
    BLOCK_M1: tl.constexpr, # dKとdVの計算に使用するブロックサイズ
    BLOCK_N1: tl.constexpr, # dKとdVの計算に使用するブロックサイズ
    BLOCK_M2: tl.constexpr,  # dQの計算に使用するブロックサイズ
    BLOCK_N2: tl.constexpr,  # dQの計算に使用するブロックサイズ
    BLK_SLICE_FACTOR: tl.constexpr, # ブロックを更に細かくスライスする際の分割数
    HEAD_DIM: tl.constexpr # アテンションヘッドの次元数
):

    #########
    # 初期化 #
    #########

    # ln(2)を定数として定義
    LN2: tl.constexpr = 0.6931471824645996

    # 現在のプログラムインスタンスIDの2次元目を取得し、バッチとヘッドの複合インデックスとする
    bhid = tl.program_id(2)

    # バッチとヘッドのオフセットを計算
    off_chz = (bhid * N_CTX).to(tl.int64)

    # オフセットを計算し、INT64にキャスト
    # ヘッダのオフセット + バッチのオフセット
    adj = (stride_h * (bhid % H) + stride_z * (bhid // H)).to(tl.int64)

    # 現在のプログラムインスタンスIDの0次元目を取得し、行もしくは列のブロックインデックスとする
    pid = tl.program_id(0)

    # 現在のプログラムインスタンスが担当するバッチとヘッドのオフセットをポインタに反映
    Q += adj
    K += adj
    V += adj
    DO += adj
    DQ += adj
    DK += adj
    DV += adj
    M += off_chz
    D += off_chz

    # ヘッドの次元インデックスの配列を作成
    offs_k = tl.arange(0, HEAD_DIM)

    ##############
    # dKとdVを計算 #
    ##############

    # 現在のプログラムインスタンスが担当するKブロック・Vブロックの開始列インデックスを計算
    start_n = pid * BLOCK_N1

    # 現在のプログラムインスタンスが担当するQブロックの開始行インデックスを計算
    start_m = start_n

    # 一度に処理するK・Vブロックサイズを更に細かくする（1/2）
    MASK_BLOCK_M1: tl.constexpr = BLOCK_M1 // BLK_SLICE_FACTOR

    # 現在のプログラムインスタンスが処理するKブロックの列オフセットを計算
    # (BLOCK_N1,)
    offs_n = start_n + tl.arange(0, BLOCK_N1)

    # dVをゼロで初期化
    # (BLOCK_N1, HEAD_DIM)
    dv = tl.zeros([BLOCK_N1, HEAD_DIM], dtype=tl.float32)

    # dKをゼロで初期化
    # (BLOCK_N1, HEAD_DIM)
    dk = tl.zeros([BLOCK_N1, HEAD_DIM], dtype=tl.float32)

    # 現在のプログラムインスタンスが処理するKブロックをSRAMにロード
    # Kポインタ + 行オフセット * トークンストライド + 列オフセット * 次元ストライド
    # (BLOCK_N1, HEAD_DIM)
    k = tl.load(K + offs_n[:, None] * stride_tok + offs_k[None, :] * stride_d)

    # 現在のプログラムインスタンスが処理するVブロックをSRAMにロード
    # Vポインタ + 行オフセット * トークンストライド + 列オフセット * 次元ストライド
    # (BLOCK_N1, HEAD_DIM)
    v = tl.load(V + offs_n[:, None] * stride_tok + offs_k[None, :] * stride_d)

    # 必要な反復回数を計算
    num_steps = BLOCK_N1 // MASK_BLOCK_M1

    # 因果マスキングが必要な対角ブロックに対してdKとdVを計算（on-diagonal）
    dk, dv = _attn_bwd_dkdv(
        dk,
        dv,
        Q,
        k,
        v,
        sm_scale,
        DO,
        M,
        D,
        stride_tok,
        stride_d,
        H,
        N_CTX,
        MASK_BLOCK_M1,
        BLOCK_N1,
        HEAD_DIM,
        start_n,
        start_m,
        num_steps,
        MASK=True # 因果マスキングを適用
    )

    # ひとつのブロック分進める（逆伝播のため、オンダイアゴナルからオフダイアゴナルに進める）
    start_m += num_steps * MASK_BLOCK_M1
    num_steps = (N_CTX - start_m) // BLOCK_M1

    # 因果マスキングが不要なブロックに対してdKとdVを計算（off-diagonal）
    dk, dv = _attn_bwd_dkdv(
        dk,
        dv,
        Q,
        k,
        v,
        sm_scale,
        DO,
        M,
        D,
        stride_tok,
        stride_d,
        H,
        N_CTX,
        BLOCK_M1,
        BLOCK_N1,
        HEAD_DIM,
        start_n,
        start_m,
        num_steps,
        MASK=False # 因果マスキングを適用しない
    )

    # dVの出力先のポインタを計算
    # ベースポインタ + 行オフセット * トークンストライド + 列オフセット * 次元ストライド
    # (BLOCK_N1, HEAD_DIM)
    dv_ptrs = DV + offs_n[:, None] * stride_tok + offs_k[None, :] * stride_d

    # dVを書き戻す
    tl.store(dv_ptrs, dv)

    # dKのスケールを戻す
    dk *= sm_scale

    # dKの出力先のポインタを計算
    # ベースポインタ + 行オフセット * トークンストライド + 列オフセット * 次元ストライド
    # (BLOCK_N1, HEAD_DIM)
    dk_ptrs = DK + offs_n[:, None] * stride_tok + offs_k[None, :] * stride_d

    # dKを書き戻す
    tl.store(dk_ptrs, dk)

    ###########
    # dQを計算 #
    ###########

    # 現在のプログラムインスタンスが処理するQブロックの開始行インデックスを計算
    start_m = pid * BLOCK_M2

    # 現在のプログラムインスタンスが処理するQブロックの終了行インデックスを計算 
    end_n = start_m + BLOCK_M2

    # 一度に処理するK・Vブロックサイズを更に細かくする（1/2）
    MASK_BLOCK_N2: tl.constexpr = BLOCK_N2 // BLK_SLICE_FACTOR

    # Qブロックの行オフセットを計算
    # (BLOCK_M2,)
    offs_m = start_m + tl.arange(0, BLOCK_M2)

    # QブロックをSRAMにロード
    # Qポインタ + 行オフセット * トークンストライド + 列オフセット * 次元ストライド
    # (BLOCK_M2, HEAD_DIM)
    q = tl.load(Q + offs_m[:, None] * stride_tok + offs_k[None, :] * stride_d)

    # dQをゼロで初期化
    # (BLOCK_M2, HEAD_DIM)
    dq = tl.zeros([BLOCK_M2, HEAD_DIM], dtype=tl.float32)

    # dOブロックをSRAMにロード
    # DOポインタ + 行オフセット * トークンストライド + 列オフセット * 次元ストライド
    # (BLOCK_M2, HEAD_DIM)
    do = tl.load(DO + offs_m[:, None] * stride_tok + offs_k[None, :] * stride_d)

    # log-sum-expテンソルをSRAMにロード
    # (BLOCK_M2,) -> (BLOCK_M2, 1)
    m = tl.load(M + offs_m)
    m = m[:, None]

    # 必要な反復回数を計算
    num_steps = BLOCK_M2 // MASK_BLOCK_N2

    # 因果マスキングが必要な対角ブロックに対してdQを計算（on-diagonal）
    dq = _attn_bwd_dq(
        dq,
        q,
        K,
        V,
        do,
        m,
        D,
        stride_tok,
        stride_d,
        H,
        N_CTX,
        BLOCK_M2,
        MASK_BLOCK_N2,
        HEAD_DIM,
        start_m,
        end_n - num_steps * MASK_BLOCK_N2,
        num_steps,
        MASK=True # 因果マスキングを適用
    )

    # ひとつのブロック分進める（逆方向に進めるのはコードの再利用性とシンプルさのため）
    end_n -= num_steps * MASK_BLOCK_N2
    num_steps = end_n // BLOCK_N2

    # 因果マスキングが不要なブロックに対してdQを計算（off-diagonal）
    dq = _attn_bwd_dq(
        dq,
        q,
        K,
        V,
        do,
        m,
        D,
        stride_tok,
        stride_d,
        H,
        N_CTX,
        BLOCK_M2,
        BLOCK_N2,
        HEAD_DIM,
        start_m,
        end_n - num_steps * BLOCK_N2,
        num_steps,
        MASK=False # 因果マスキングを適用しない
    )

    ###############
    # dQの書き戻し #
    ###############

    # dQの出力先のポインタを計算
    # ベースポインタ + 行オフセット * トークンストライド + 列オフセット * 次元ストライド
    # (BLOCK_M2, HEAD_DIM)
    dq_ptrs = DQ + offs_m[:, None] * stride_tok + offs_k[None, :] * stride_d

    # dQをスケーリング（順伝播でexp2を使用したため）
    dq *= LN2

    # dQを書き戻す
    tl.store(dq_ptrs, dq)

### Attention層

In [ ]:
class _attention(torch.autograd.Function):

    @staticmethod
    def forward(
        ctx, # PyTorchで自動的に渡されるコンテキストオブジェクト
        q, # クエリテンソル (Z, H, N_CTX, HEAD_DIM)
        k, # キーテンソル (Z, H, N_CTX, HEAD_DIM)
        v, # バリューテンソル (Z, H, N_CTX, HEAD_DIM)
        causal, # 因果マスキングを適用するかどうかのフラグ
        sm_scale, # スケーリング係数
        warp_specialize=True # Tritonの最適化フラグ
    ):
        logger.info(f"順伝搬開始 {q.shape=}{q.dtype=}, {k.shape=}, {k.dtype=}, {v.shape=}, {v.dtype=}, {causal=}, {sm_scale=}, {warp_specialize=}")

        #########
        # 初期化 #
        #########

        # Qのヘッド次元を取得 64
        HEAD_DIM_Q = q.shape[-1]
        logger.debug(f"{HEAD_DIM_Q=}")

        # Kのヘッド次元を取得 64
        HEAD_DIM_K = k.shape[-1]
        logger.debug(f"{HEAD_DIM_K=}")

        # Vのヘッド次元を取得 64
        HEAD_DIM_V = v.shape[-1]
        logger.debug(f"{HEAD_DIM_V=}")

        # Q・K・Vのヘッド次元が同じであることを検証
        assert HEAD_DIM_Q == HEAD_DIM_K and HEAD_DIM_K == HEAD_DIM_V

        # Kのヘッド次元がサポートされている値であることを検証
        assert HEAD_DIM_K in {16, 32, 64, 128, 256}

        # 出力テンソルOを初期化 (1, 2, 128, 64)
        # (Z, H, N_CTX, HEAD_DIM)
        o = torch.empty_like(q)
        logger.debug(f"{o.shape=}, {o.dtype=}")

        # ステージの設定 3
        # 因果マスクが有効な場合はステージ3、そうでなければステージ1
        stage = 3 if causal else 1
        logger.debug(f"{stage=}")

        extra_kern_args = {}

        # AMDの場合 False
        if is_hip():
            waves_per_eu = 3 if HEAD_DIM_K <= 64 else 2
            extra_kern_args = {"waves_per_eu": waves_per_eu, "allow_flush_denorm": True}
        logger.debug(f"{extra_kern_args=}")

        # 生のアテンションスコアの要素の最大値mを格納するテンソルMを初期化 (1, 2, 128)
        # (Z, H, N_CTX)
        M = torch.empty(
            (q.shape[0], q.shape[1], q.shape[2]),
            device=q.device,
            dtype=torch.float32
        )
        logger.debug(f"{M.shape=}")

        ####################
        # ディスクリプタの準備 #
        ####################

        # 新しいGPUの場合 True
        if supports_host_descriptor() and not (is_hopper() and warp_specialize):
            # バッチサイズ、ヘッド数、コンテキスト長を掛け合わせてフラット化
            # 1 * 2 * 128 = 256
            y_dim = q.shape[0] * q.shape[1] * q.shape[2]
            logger.debug(f"{y_dim=}")

            dummy_block = [1, 1]

            # Qのテンソルディスクリプタを作成
            desc_q = TensorDescriptor(
                q,
                shape=[y_dim, HEAD_DIM_K], # (Z * H * N_CTX, HEAD_DIM)
                strides=[HEAD_DIM_K, 1], # (HEAD_DIM, 1)
                block_shape=dummy_block
            )
            logger.debug(f"Qのディスクリプタを作成")

            # Vのテンソルディスクリプタを作成
            if q.dtype == torch.float8_e5m2:
                # Qのデータ型がFP8の場合は、メモリレイアウトが異なるので注意
                desc_v = TensorDescriptor(
                    v,
                    shape=[HEAD_DIM_K, y_dim], # (HEAD_DIM, Z * H * N_CTX)
                    strides=[q.shape[2], 1], # (N_CTX, 1)
                    block_shape=dummy_block
                )
                logger.debug(f"FP8用のVのディスクリプタを作成")
            else:
                desc_v = TensorDescriptor(
                    v,
                    shape=[y_dim, HEAD_DIM_K], # (Z * H * N_CTX, HEAD_DIM)
                    strides=[HEAD_DIM_K, 1], # (HEAD_DIM, 1)
                    block_shape=dummy_block
                )
                logger.debug(f"Vのディスクリプタを作成")

            # Kのテンソルディスクリプタを作成
            desc_k = TensorDescriptor(
                k,
                shape=[y_dim, HEAD_DIM_K], # (Z * H * N_CTX, HEAD_DIM)
                strides=[HEAD_DIM_K, 1], # (HEAD_DIM, 1)
                block_shape=dummy_block
            )
            logger.debug(f"Kのディスクリプタを作成")

            # Oのテンソルディスクリプタを作成
            desc_o = TensorDescriptor(
                o,
                shape=[y_dim, HEAD_DIM_K], # (Z * H * N_CTX, HEAD_DIM)
                strides=[HEAD_DIM_K, 1], # (HEAD_DIM, 1)
                block_shape=dummy_block
            )
            logger.debug(f"Oのディスクリプタを作成")
        else:
            desc_q = q
            desc_v = v
            desc_k = k
            desc_o = o
            logger.debug(f"ポインタを準備 {desc_q.shape=}, {desc_k.shape=}, {desc_v.shape=}, {desc_o.shape=}")

        ################
        # カーネルの起動 #
        ################

        def alloc_fn(size: int, align: int, _):
            return torch.empty(size, dtype=torch.int8, device="cuda")

        triton.set_allocator(alloc_fn)

        # 起動グリッドを定義
        # Qブロックのヘッドごとにプログラムインスタンスを割り当てる
        def grid(META):
            return (
                triton.cdiv(q.shape[2], META["BLOCK_M"]), # N_CTX / BLOCK_M = Qブロックの総数
                q.shape[0] * q.shape[1], # バッチサイズ * ヘッド数 = ヘッドの総数
                1 
            )
        logger.debug(f"起動グリッドを準備")

        ctx.grid = grid

        if is_blackwell() and warp_specialize:
            if HEAD_DIM_K == 128 and q.dtype == torch.float16:
                extra_kern_args["maxnreg"] = 168
            else:
                extra_kern_args["maxnreg"] = 80

        logger.debug(f"_attn_fwdカーネルの実行")
        _attn_fwd[grid](
            sm_scale,
            M,
            q.shape[0],
            q.shape[1],
            desc_q,
            desc_k,
            desc_v,
            desc_o,
            N_CTX=q.shape[2],
            HEAD_DIM=HEAD_DIM_K,
            FP8_OUTPUT=q.dtype==torch.float8_e5m2,
            STAGE=stage,
            warp_specialize=warp_specialize,
            IS_HOPPER=is_hopper(),
            **extra_kern_args
        )

        #########
        # 後処理 #
        #########

        # 逆伝搬で使用する値を保存
        ctx.save_for_backward(q, k, v, o, M)
        ctx.sm_scale = sm_scale
        ctx.HEAD_DIM = HEAD_DIM_K
        ctx.causal = causal

        logger.info(f"順伝搬終了 {o.shape=}, {o.dtype=}")
        return o 

    @staticmethod
    def backward(
        ctx, # forwardで保存した値を保持するコンテキストオブジェクト
        do # 出力Oの勾配テンソル
    ):
        logger.info(f"逆伝搬開始 {do.shape=}, {do.dtype=}")

        #########
        # 初期化 #
        #########

        # 保存したテンソルを取得
        q, k, v, o, M = ctx.saved_tensors

        logger.debug(f"{q.shape=}, {k.shape=}, {v.shape=}, {o.shape=}, {M.shape=}")

        # dOのメモリが連続していることを検証
        assert do.is_contiguous()

        # 全てのテンソルのストライドが同じであることを検証
        assert q.stride() == k.stride() == v.stride() == o.stride() == do.stride()

        # 出力先の勾配テンソルを初期化
        dq = torch.empty_like(q)
        dk = torch.empty_like(k)
        dv = torch.empty_like(v)

        # 1, 2, 128
        BATCH, N_HEAD, N_CTX = q.shape[:3]
        logger.debug(f"{BATCH=}, {N_HEAD=}, {N_CTX=}")

        PRE_BLOCK = 128
        logger.debug(f"{PRE_BLOCK=}")

        NUM_WARPS, NUM_STAGES = 4, 5
        logger.debug(f"{NUM_WARPS=}, {NUM_STAGES=}")

        BLOCK_M1, BLOCK_N1, BLOCK_M2, BLOCK_N2 = 32, 128, 128, 32
        logger.debug(f"{BLOCK_M1=}, {BLOCK_N1=}, {BLOCK_M2=}, {BLOCK_N2=}")

        BLK_SLICE_FACTOR = 2
        logger.debug(f"{BLK_SLICE_FACTOR=}")

        RCP_LN2 = 1.4426950408889634  # = 1.0 / ln(2)
        logger.debug(f"{RCP_LN2=}")

        # Kを事前スケーリング
        # アテンションウェイトの再計算でexp2を使用するため
        arg_k = k
        arg_k = arg_k * (ctx.sm_scale * RCP_LN2)
        logger.debug(f"{arg_k.shape=}")

        ###################################
        # _attn_bwd_preprocessカーネルの実行 #
        ###################################

        PRE_BLOCK = 128
        logger.debug(f"{PRE_BLOCK=}")
        
        # N_CTXがPRE_BLOCKの倍数であることを検証
        assert N_CTX % PRE_BLOCK == 0

        # 2次元の起動グリッドを定義 (1, 2)
        pre_grid = (
            N_CTX // PRE_BLOCK, # N_CTX / PRE_BLOCK = Qブロックの総数
            BATCH * N_HEAD # バッチサイズ * ヘッド数 = ヘッドの総数
        )
        logger.debug(f"{pre_grid=}")

        # preprocessカーネルの計算結果を格納するテンソルを初期化 (1, 2, 128)
        # (Z, H, N_CTX)
        delta = torch.empty_like(M)
        logger.debug(f"{delta.shape=}")

        logger.debug(f"_attn_bwd_preprocessカーネルの実行（Deltaを計算）")
        _attn_bwd_preprocess[pre_grid](
            o,
            do,
            delta,
            BATCH,
            N_HEAD,
            N_CTX,
            BLOCK_M=PRE_BLOCK,
            HEAD_DIM=ctx.HEAD_DIM
        )

        #########################
        # _attn_bwdカーネルの実行 #
        #########################

        # 3次元の起動グリッドを定義 (1, 1, 2)
        grid = (
            N_CTX // BLOCK_N1, # N_CTX / BLOCK_N1 = Kブロックの総数
            1,
            BATCH * N_HEAD # バッチサイズ * ヘッド数 = ヘッドの総数
        )
        logger.debug(f"{grid=}")

        logger.debug(f"_attn_bwdカーネルの実行（dQ・dK・dVを計算）")
        _attn_bwd[grid](
            q,
            arg_k,
            v,
            ctx.sm_scale,
            do,
            dq,
            dk,
            dv,
            M,
            delta,
            q.stride(0),
            q.stride(1),
            q.stride(2),
            q.stride(3),
            N_HEAD,
            N_CTX,
            BLOCK_M1=BLOCK_M1,
            BLOCK_N1=BLOCK_N1,
            BLOCK_M2=BLOCK_M2,
            BLOCK_N2=BLOCK_N2,
            BLK_SLICE_FACTOR=BLK_SLICE_FACTOR,
            HEAD_DIM=ctx.HEAD_DIM,
            num_warps=NUM_WARPS,
            num_stages=NUM_STAGES
        )

        logger.info(f"逆伝搬終了 {dq.shape=}, {dq.dtype=}, {dk.shape=}, {dk.dtype=}, {dv.shape=}, {dv.dtype=}")
        return dq, dk, dv, None, None, None, None

In [20]:
attention = _attention.apply
attention

<bound method Function.apply of <class '__main__._attention'>>

### 検証

In [21]:
TORCH_HAS_FP8 = hasattr(torch, 'float8_e5m2')
TORCH_HAS_FP8

True

In [ ]:
@pytest.mark.parametrize("Z", [1, 4])
@pytest.mark.parametrize("H", [2, 48])
@pytest.mark.parametrize("N_CTX", [128, 1024, (2 if is_hip() else 4) * 1024])
@pytest.mark.parametrize("HEAD_DIM", [64, 128])
@pytest.mark.parametrize("causal", [True])  # FIXME: Non-causal tests do not pass at the moment.
@pytest.mark.parametrize("warp_specialize", [False, True] if is_blackwell() else [False])
@pytest.mark.parametrize("mode", ["fwd", "bwd"])
@pytest.mark.parametrize("provider", ["triton-fp16"] + (["triton-fp8"] if TORCH_HAS_FP8 else []))
def test_op(Z, H, N_CTX, HEAD_DIM, causal, warp_specialize, mode, provider, dtype=torch.float16):
    logger.info(f"テスト開始 Z={Z}, H={H}, N_CTX={N_CTX}, HEAD_DIM={HEAD_DIM}, causal={causal}, warp_specialize={warp_specialize}, mode={mode}, provider={provider}, dtype={dtype}")

    if mode == "fwd" and "fp16" in provider:
        pytest.skip("Avoid running the forward computation twice.")

    if mode == "bwd" and "fp8" in provider:
        pytest.skip("Backward pass with FP8 is not supported.")

    torch.manual_seed(20)

    q = (torch.empty((Z, H, N_CTX, HEAD_DIM), dtype=dtype, device=DEVICE).normal_(mean=0.0, std=0.5).requires_grad_())
    logger.debug(f"{q.shape=}")

    k = (torch.empty((Z, H, N_CTX, HEAD_DIM), dtype=dtype, device=DEVICE).normal_(mean=0.0, std=0.5).requires_grad_())
    logger.debug(f"{k.shape=}")

    v = (torch.empty((Z, H, N_CTX, HEAD_DIM), dtype=dtype, device=DEVICE).normal_(mean=0.0, std=0.5).requires_grad_())
    logger.debug(f"{v.shape=}")

    sm_scale = 0.5

    ###########################################
    # PyTorchの標準関数を使用してアテンションを計算 #
    ###########################################

    ref_dtype = dtype

    if mode == "fwd" and "fp8" in provider:
        ref_dtype = torch.float32

    q = q.to(ref_dtype)
    k = k.to(ref_dtype)
    v = v.to(ref_dtype)

    # 因果マスクの作成
    M = torch.tril(torch.ones((N_CTX, N_CTX), device=DEVICE))
    logger.debug(f"{M.shape=}")

    p = torch.matmul(q, k.transpose(2, 3)) * sm_scale
    logger.debug(f"{p.shape=}")

    if causal:
        p[:, :, M == 0] = float("-inf")
        logger.debug("因果マスクを適用")

    p = torch.softmax(p.float(), dim=-1)
    logger.debug("ソフトマックスを適用")

    p = p.to(ref_dtype)
    # p = torch.exp(p)

    ref_out = torch.matmul(p, v).half()
    logger.debug(f"バリューを集約 {ref_out.shape=}")

    if mode == "bwd":
        dout = torch.randn_like(q)
        ref_out.backward(dout)
        ref_dv, v.grad = v.grad.clone(), None
        ref_dk, k.grad = k.grad.clone(), None
        ref_dq, q.grad = q.grad.clone(), None

    #############################
    # Tritonによるアテンション計算 #
    #############################

    if mode == "fwd" and "fp8" in provider:
        q = q.to(torch.float8_e5m2)
        k = k.to(torch.float8_e5m2)
        v = v.permute(0, 1, 3, 2).contiguous()
        v = v.permute(0, 1, 3, 2)
        v = v.to(torch.float8_e5m2)

    tri_out = attention(q, k, v, causal, sm_scale, warp_specialize).half()
    logger.debug(f"Tritonによるアテンション計算 {tri_out.shape=}")

    if mode == "fwd":
        atol = 3 if "fp8" in provider else 1e-2
        torch.testing.assert_close(tri_out, ref_out, atol=atol, rtol=0)
        return

    tri_out.backward(dout)
    logger.debug(f"Tritionによる逆伝播計算 {tri_out.shape=}")

    tri_dv, v.grad = v.grad.clone(), None
    tri_dk, k.grad = k.grad.clone(), None
    tri_dq, q.grad = q.grad.clone(), None

    ################
    # 計算結果を比較 #
    ################

    torch.testing.assert_close(tri_out, ref_out, atol=1e-2, rtol=0)
    rtol = 0.0

    # Relative tolerance workaround for known hardware limitation of CDNA2 GPU.
    # For details see https://pytorch.org/docs/stable/notes/numerical_accuracy.html#reduced-precision-fp16-and-bf16-gemms-and-convolutions-on-amd-instinct-mi200-devices
    if torch.version.hip is not None and \
        triton.runtime.driver.active.get_current_target().arch == "gfx90a":
        rtol = 1e-2

    torch.testing.assert_close(tri_dv, ref_dv, atol=1e-2, rtol=rtol)
    torch.testing.assert_close(tri_dk, ref_dk, atol=1e-2, rtol=rtol)
    torch.testing.assert_close(tri_dq, ref_dq, atol=1e-2, rtol=rtol)

In [ ]:
test_op(
    Z=1,
    H=2,
    N_CTX=128,
    HEAD_DIM=32,
    causal=True,
    warp_specialize=False,
    mode="fwd",
    provider="triton-fp8",
)

🟩 テスト開始 Z=1, H=2, N_CTX=128, HEAD_DIM=64, causal=True, warp_specialize=False, mode=fwd, provider=triton-fp8, dtype=torch.float16
🟦 q.shape=torch.Size([1, 2, 128, 64])
🟦 k.shape=torch.Size([1, 2, 128, 64])
🟦 v.shape=torch.Size([1, 2, 128, 64])
🟦 M.shape=torch.Size([128, 128])
🟦 p.shape=torch.Size([1, 2, 128, 128])
🟦 因果マスクを適用
🟦 ソフトマックスを適用
🟦 バリューを集約 ref_out.shape=torch.Size([1, 2, 128, 64])
🟩 順伝搬開始 q.shape=torch.Size([1, 2, 128, 64])q.dtype=torch.float8_e5m2, k.shape=torch.Size([1, 2, 128, 64]), k.dtype=torch.float8_e5m2, v.shape=torch.Size([1, 2, 128, 64]), v.dtype=torch.float8_e5m2, causal=True, sm_scale=0.5, warp_specialize=False
🟦 HEAD_DIM_Q=64
🟦 HEAD_DIM_K=64
🟦 HEAD_DIM_V=64
🟦 o.shape=torch.Size([1, 2, 128, 64]), o.dtype=torch.float8_e5m2
🟦 stage=3
🟦 extra_kern_args={}
🟦 M.shape=torch.Size([1, 2, 128])
🟦 y_dim=256
🟦 Qのディスクリプタを作成
🟦 FP8用のVのディスクリプタを作成
🟦 Kのディスクリプタを作成
🟦 Oのディスクリプタを作成
🟦 起動グリッドを準備
🟦 _attn_fwdカーネルの実行
🟩 順伝搬終了 o.shape=torch.Size([1, 2, 128, 64]), o.dtype=torch.float8_e5m2
🟦 Tri

In [ ]:
test_op(
    Z=1,
    H=2,
    N_CTX=128,
    HEAD_DIM=32,
    causal=True,
    warp_specialize=False,
    mode="bwd",
    provider="triton-fp16",
)

🟩 テスト開始 Z=1, H=2, N_CTX=128, HEAD_DIM=64, causal=True, warp_specialize=False, mode=bwd, provider=triton-fp16, dtype=torch.float16
🟦 q.shape=torch.Size([1, 2, 128, 64])
🟦 k.shape=torch.Size([1, 2, 128, 64])
🟦 v.shape=torch.Size([1, 2, 128, 64])
🟦 M.shape=torch.Size([128, 128])
🟦 p.shape=torch.Size([1, 2, 128, 128])
🟦 因果マスクを適用
🟦 ソフトマックスを適用
🟦 バリューを集約 ref_out.shape=torch.Size([1, 2, 128, 64])
/opt/miniconda/envs/py312/lib/python3.12/site-packages/torch/autograd/graph.py:829: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /pytorch/aten/src/ATen/cuda/CublasHandlePool.cpp:179.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
🟩 順伝搬開始 q.shape=torch.Size([1, 2, 128, 64])q.dtype=torch.float16, k.shape=torch.Size([1, 2, 128, 64]), k.dtype=torch.float16, v.shape=torch.Size([1, 2, 128, 64]), v.dtype=torch.float16, causal=True, sm_scale=0.5, warp_spe

### ベンチマーク

In [ ]:
logger.setLevel(logging.WARNING)

In [ ]:
try:
    from flash_attn.flash_attn_interface import \
        flash_attn_qkvpacked_func as flash_attn_func
    HAS_FLASH = True
except BaseException:
    HAS_FLASH = False

HAS_FLASH

In [ ]:
TORCH_HAS_FP8 = hasattr(torch, 'float8_e5m2')
TORCH_HAS_FP8

In [ ]:
BATCH, N_HEADS = 4, 32

In [ ]:
# vary seq length for fixed head and batch=4
configs = []
for HEAD_DIM in [64, 128]:
    for mode in ["fwd", "bwd"]:
        for causal in [True, False]:
            # Enable warpspec for causal fwd on Hopper
            enable_ws = mode == "fwd" and (is_blackwell() or (is_hopper() and not causal))
            for warp_specialize in [False, True] if enable_ws else [False]:
                configs.append(
                    triton.testing.Benchmark(
                        x_names=["N_CTX"],
                        x_vals=[2**i for i in range(10, 15)],
                        line_arg="provider",
                        line_vals=["triton-fp16"] + (["triton-fp8"] if TORCH_HAS_FP8 else []) +
                        (["flash"] if HAS_FLASH else []),
                        line_names=["Triton [FP16]"] + (["Triton [FP8]"] if TORCH_HAS_FP8 else []) +
                        (["Flash-2"] if HAS_FLASH else []),
                        styles=[("red", "-"), ("blue", "-"), ("green", "-")],
                        ylabel="TFLOPS",
                        plot_name=
                        f"fused-attention-batch{BATCH}-head{N_HEADS}-d{HEAD_DIM}-{mode}-causal={causal}-warp_specialize={warp_specialize}",
                        args={
                            "H": N_HEADS,
                            "BATCH": BATCH,
                            "HEAD_DIM": HEAD_DIM,
                            "mode": mode,
                            "causal": causal,
                            "warp_specialize": warp_specialize,
                        },
                    ))

configs

In [ ]:
@triton.testing.perf_report(configs)
def bench_flash_attention(BATCH, H, N_CTX, HEAD_DIM, causal, warp_specialize, mode, provider, device=DEVICE):
    assert mode in ["fwd", "bwd"]
    dtype = torch.float16
    if "triton" in provider:
        q = torch.randn((BATCH, H, N_CTX, HEAD_DIM), dtype=dtype, device=device, requires_grad=True)
        k = torch.randn((BATCH, H, N_CTX, HEAD_DIM), dtype=dtype, device=device, requires_grad=True)
        v = torch.randn((BATCH, H, N_CTX, HEAD_DIM), dtype=dtype, device=device, requires_grad=True)
        if mode == "fwd" and "fp8" in provider:
            q = q.to(torch.float8_e5m2)
            k = k.to(torch.float8_e5m2)
            v = v.permute(0, 1, 3, 2).contiguous()
            v = v.permute(0, 1, 3, 2)
            v = v.to(torch.float8_e5m2)
        sm_scale = 1.3
        fn = lambda: attention(q, k, v, causal, sm_scale, warp_specialize)
        if mode == "bwd":
            o = fn()
            do = torch.randn_like(o)
            fn = lambda: o.backward(do, retain_graph=True)
        ms = triton.testing.do_bench(fn)

    if provider == "flash":
        qkv = torch.randn((BATCH, N_CTX, 3, H, HEAD_DIM), dtype=dtype, device=device, requires_grad=True)
        fn = lambda: flash_attn_func(qkv, causal=causal)
        if mode == "bwd":
            o = fn()
            do = torch.randn_like(o)
            fn = lambda: o.backward(do, retain_graph=True)
        ms = triton.testing.do_bench(fn)
    flops_per_matmul = 2.0 * BATCH * H * N_CTX * N_CTX * HEAD_DIM
    total_flops = 2 * flops_per_matmul
    if causal:
        total_flops *= 0.5
    if mode == "bwd":
        total_flops *= 2.5  # 2.0(bwd) + 0.5(recompute)
    return total_flops * 1e-12 / (ms * 1e-3)

# only works on post-Ampere GPUs right now
bench_flash_attention.run(save_path=".", print_data=True)